In [4]:
from google.colab import files

uploaded = files.upload()  # Chọn file dữ liệu và stopword để upload


Saving stopword.txt to stopword.txt


In [7]:
import pandas as pd

# Đọc file CSV
df = pd.read_csv("data.csv")

# Xem thông tin dữ liệu
df.head()


,content,label,start
0,Áo bao đẹp ạ!,POS,5
1,Tuyệt vời,POS,5
2,2day ao khong giong trong,NEG,1
3,"Mùi thơm,bôi lên da mềm da",POS,5
4,"Vải đẹp, dày dặn",POS,5


In [10]:
!pip install torch torchvision torchaudio transformers underthesea vncorenlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 43.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 75.1 MB/s eta 0:00:00
   

In [13]:
import pandas as pd
import re
from underthesea import word_tokenize

# Đọc dữ liệu
df = pd.read_csv("data.csv")

# Đọc danh sách stopwords
with open("stopwords.txt", "r", encoding="utf-8") as f:
    stopwords = set(f.read().splitlines())

# Hàm tiền xử lý văn bản
def clean_text(text):
    text = text.lower()  # Chuyển về chữ thường
    text = re.sub(r"[^\w\s]", "", text)  # Xóa dấu câu
    text = word_tokenize(text, format="text")  # Tách từ tiếng Việt
    words = text.split()
    words = [word for word in words if word not in stopwords or word == "không"]  # Giữ lại từ "không"
    return " ".join(words)

# Áp dụng tiền xử lý
df["content_clean"] = df["content"].astype(str).apply(clean_text)

# Chuyển nhãn cảm xúc
def convert_label(score):
    if score >= 4:
        return "positive"
    elif score == 3:
        return "neutral"
    else:
        return "negative"

df["sentiment"] = df["start"].apply(convert_label)

df.head()


,content,label,start,content_clean,sentiment
0,Áo bao đẹp ạ!,POS,5,áo bao đẹp,positive
1,Tuyệt vời,POS,5,tuyệt_vời,positive
2,2day ao khong giong trong,NEG,1,2 day ao khong_giong,negative
3,"Mùi thơm,bôi lên da mềm da",POS,5,mùi thơmbôi da mềm da,positive
4,"Vải đẹp, dày dặn",POS,5,vải đẹp dày dặn,positive


In [14]:
from transformers import AutoTokenizer

# Load tokenizer của PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

# Mã hóa dữ liệu thành token
def encode_text(text):
    return tokenizer(text, truncation=True, padding="max_length", max_length=256, return_tensors="pt")

# Mã hóa toàn bộ tập dữ liệu
encoded_texts = [encode_text(text) for text in df["content_clean"]]


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

In [16]:
!pip install torch torchvision torchaudio


In [17]:
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModel, AdamW

# Load PhoBERT model
phobert = AutoModel.from_pretrained("vinai/phobert-base")

# Xây dựng tập dữ liệu
class SentimentDataset(Dataset):
    def __init__(self, texts, labels):
        self.texts = texts
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.texts[idx], self.labels[idx]

# Chuyển đổi nhãn thành số (0: negative, 1: neutral, 2: positive)
label_map = {"negative": 0, "neutral": 1, "positive": 2}
df["label_id"] = df["sentiment"].map(label_map)

# Chia tập train/test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(encoded_texts, df["label_id"], test_size=0.2, random_state=42)

# Tạo DataLoader
train_dataset = SentimentDataset(X_train, y_train.tolist())
test_dataset = SentimentDataset(X_test, y_test.tolist())

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [18]:
import torch  # Cần import trước khi dùng torch.device
import torch.nn as nn
import torch.optim as optim

# Mô hình phân loại
class PhoBERTClassifier(nn.Module):
    def __init__(self, phobert_model):
        super(PhoBERTClassifier, self).__init__()
        self.phobert = phobert_model
        self.fc = nn.Linear(768, 3)  # 3 lớp output (negative, neutral, positive)

    def forward(self, input_ids, attention_mask):
        outputs = self.phobert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[0][:, 0, :]  # Lấy vector của token đầu tiên [CLS]
        return self.fc(pooled_output)

# Khởi tạo mô hình
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PhoBERTClassifier(phobert).to(device)

# Hàm tối ưu & loss function
optimizer = AdamW(model.parameters(), lr=2e-5)
criterion = nn.CrossEntropyLoss()

# Huấn luyện
epochs = 3
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        input_ids = batch[0]["input_ids"].squeeze(1).to(device)
        attention_mask = batch[0]["attention_mask"].squeeze(1).to(device)
        labels = torch.tensor(batch[1]).to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}")


/usr/local/lib/python3.11/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-18-254f053559db>:33: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch[1]).to(device)


Epoch 1, Loss: 0.6041817095957208
Epoch 2, Loss: 0.5189703235761496
Epoch 3, Loss: 0.48463446372683006


In [19]:
from sklearn.metrics import accuracy_score, classification_report

model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch[0]["input_ids"].squeeze(1).to(device)
        attention_mask = batch[0]["attention_mask"].squeeze(1).to(device)
        labels = torch.tensor(batch[1]).to(device)

        outputs = model(input_ids, attention_mask)
        preds = torch.argmax(outputs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# In kết quả
print("Accuracy:", accuracy_score(all_labels, all_preds))
print(classification_report(all_labels, all_preds, target_names=["negative", "neutral", "positive"]))

<ipython-input-19-dbb50455b812>:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(batch[1]).to(device)


Accuracy: 0.7860775588048315
              precision    recall  f1-score   support

    negative       0.66      0.84      0.74      1333
     neutral       0.45      0.17      0.25       978
    positive       0.87      0.92      0.89      3981

    accuracy                           0.79      6292
   macro avg       0.66      0.64      0.63      6292
weighted avg       0.76      0.79      0.76      6292



In [37]:
def predict_sentiment(text):
    text_clean = clean_text(text)
    encoded_text = encode_text(text_clean)

    input_ids = encoded_text["input_ids"].to(device)
    attention_mask = encoded_text["attention_mask"].to(device)

    model.eval()
    with torch.no_grad():
        output = model(input_ids, attention_mask)
        pred = torch.argmax(output, dim=1).cpu().numpy()[0]

    return ["negative", "neutral", "positive"][pred]

# Kiểm tra
print(predict_sentiment("Sản phẩm tồi!"))
print(predict_sentiment("Sản phẩm không tồi!"))
print(predict_sentiment("Áo hơi bé!"))
print(predict_sentiment("Size áo bị lỗi!"))
print(predict_sentiment("Sản phẩm rất tuyệt vời!"))
print(predict_sentiment("Mình đặt hộ ngiu nên hong coá hình ảnh , túi đẹp, shop hỗ trợ giao nhanh, có hộp nhìn đẹp - sang lắm ạ, thankiu shop"))
print(predict_sentiment("Hàng xước và bẩn không thể chấp nhận đc. Trả hàng thì ship kh lấy hàng. Cho 1* vẫn còn nhiều!"))
print(predict_sentiment("Shop làm ăn thiếu trách nhiệm!!"))
print(predict_sentiment("Hàng bình thường"))
print(predict_sentiment("Túi đẹp nhưng đường chỉ may xấu, chỉ thừa nhiều"))
print(predict_sentiment("túi xinh lắm nha, mỗi tội hơi to một tí thui, giá rẻ, mua đi mn ơi đẹp lắm nhaaa"))

negative
positive
negative
negative
positive
positive
negative
negative
neutral
negative
positive


In [41]:
import torch

model_path = "phobert_sentiment_model.pth"
torch.save(model.state_dict(), model_path)


In [42]:
from google.colab import files
files.download(model_path)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>